# Imports

In [ ]:
exec(open('setup.py').read())

from notebooks import *
from langgraph.constants import START, END

# Load Prompts

In [ ]:
# Load prompts from markdown files
from src.utils.prompt_loader import get_prompt

# Load all prompts
error_handler_prompt = get_prompt("error_handler")
input_guard_rail_prompt = get_prompt("input_guard_rail") 
agentic_node_prompt = get_prompt("agentic_node")

# Definitions

In [ ]:
# model definition with prompts from markdown files
agent_1 = models.GeminiModel(
    model="gemini-2.5-flash", 
    prompt=agentic_node_prompt
)

input_guard_rail_model = models.GeminiModel(
    model="gemini-2.5-flash", 
    prompt=input_guard_rail_prompt
)

error_handler_model = models.GeminiModel(
    model="gemini-2.5-flash", 
    prompt=error_handler_prompt
)

# node definition
EXIT_ZONE = END

input_guard_rail = nodes.InputGuardRail(
    routing_options={"next_node": "agentic_node", "end": "error_handler"},
    model=input_guard_rail_model,
)

agentic_node = nodes.AgenticNode(
    routing_options={"next_node": EXIT_ZONE, "end": "error_handler"},
    model=agent_1,
    tools=[],
)

error_handler = nodes.ErrorHandlerNode(
    routing_options={"end": EXIT_ZONE},
    model=error_handler_model,
)

# Graph Creation

In [ ]:
from langgraph.graph import StateGraph
from src.data_models.graph_state import CarSystemState

# Create the graph with state_schema parameter
workflow = StateGraph(state_schema=CarSystemState)

# Add nodes to the graph
workflow.add_node("input_guard_rail", input_guard_rail)
workflow.add_node("agentic_node", agentic_node)
workflow.add_node("error_handler", error_handler)

# Define the flow with conditional routing
# Os nodes usam Command para determinar o próximo node baseado no estado
workflow.add_edge(START, "input_guard_rail")

# Os nodes se roteiam automaticamente baseado nos routing_options e no estado
# Se error_message estiver presente, vão para error_handler
# Caso contrário, seguem o fluxo normal

# Compile the graph
app = workflow.compile()

# Test Graph Execution

In [ ]:
# Alternative: Synchronous test (if async has issues)
def test_graph_sync():
    """Synchronous version of the graph test."""
    
    # Create a test state using CarSystemState
    test_input = CarSystemState(
        messages=[HumanMessage(content="Meu carro não está ligando. Quando viro a chave, só faz um clique. O que pode ser?")],
        user_input=None,
        car_data=None,
        current_node=None,
        processing_status=None,
        analysis_result=None,
        recommendations=None,
        error_message=None,
        context=None,
    )
    
    print("🔧 Testing graph execution (synchronous)...")
    print(f"Input message: {test_input['messages'][0].content}")
    print("-" * 60)
    
    try:
        # Run the graph synchronously
        result = app.invoke(test_input)
        
        print("✅ Synchronous graph execution successful!")
        print("\n📊 Final State:")
        print("-" * 30)
        
        # Display the results
        for key, value in result.items():
            if value is not None:
                if key == "messages":
                    print(f"📝 {key}: {len(value)} messages")
                    for i, msg in enumerate(value):
                        print(f"   [{i}] {type(msg).__name__}: {msg.content[:80]}...")
                elif isinstance(value, str) and len(value) > 80:
                    print(f"📄 {key}: {value[:80]}...")
                else:
                    print(f"🔹 {key}: {value}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error during synchronous execution: {e}")
        print(f"Error type: {type(e).__name__}")
        import traceback
        print(f"Traceback: {traceback.format_exc()}")
        return None

print("\n" + "="*60)
print("🔄 Running synchronous version...")
sync_result = test_graph_sync()

if sync_result:
    print(f"\n🎉 Sync test completed! Final state has {len([k for k, v in sync_result.items() if v is not None])} populated fields.")
else:
    print("\n❌ Sync test also failed. Check configuration.")